In [9]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch, cm

In [23]:
cm

28.346456692913385

In [44]:
text = """
Simple Strike
	melee attack,
		+1 vs defense, lethal
	distract 2
	run 3
	"""

keywords = {
#     'stubborness': '' 
    '\t': '    ',
    'stubborness': '[#]',
    'perception': '(O)',
    'defense': 'AC'
}

def process(line):
    line = line[1:]
    for word, symbol in keywords.items():
        line = line.replace(word, symbol)
    return line


class Card:
    def __init__(self, raw):
        lines = raw.strip().split('\n')
        self.title = lines[0]
        self.lines = [process(line) for line in lines[1:]]
        
        print(self.title)
        print(self.lines)
        
    def render(self, c, xoffset, yoffset):
        offset = 0
        def write(text='', fontsize=14):
            nonlocal yoffset
            c.setFont('Helvetica', fontsize)            
            if text.startswith(keywords['\t']):
                yoffset += 6

            c.drawString(xoffset, yoffset+offset, text)
            yoffset -= .8*cm
        
        write(self.title, 20)
        yoffset -= .2*cm
        xoffset += .2*cm
        for line in self.lines:
            write(line)

# c = canvas.Canvas('test.pdf')
# Card(text).render(50, 200)
# c.showPage()
# c.save()

In [58]:

left_margin = .25*inch
width = 8*inch
height = 11*inch

rows = 4
cols = 4

class Page:    
    def __init__(self, name=None):
        self.c = canvas.Canvas(name)
        c.line(0, 0, width, height)
        
        self.row = 0
        self.col = 0

    def proceed(self):
        self.col += 1
        if self.col == cols:
            self.col = 0
            self.row += 1
            if self.row == rows:
                raise StopIterationError()
    
    def render(self, card):
        card.render(self.c, 
                    xoffset = width * (self.col/cols) + left_margin,
                    yoffset = height * ((rows-self.row)/rows))
        self.proceed()
        

# c = canvas.Canvas('out.pdf')


page = Page('out.pdf')

card = Card(text)

for _ in range(6):
    page.render(card)

page.c.showPage()
page.c.save()
        

Simple Strike
['melee attack,', '    +1 vs AC, lethal', 'distract 2', 'run 3']
